In [1]:
"""

@author: bkearney
email: bkearne5@uncc.edu

Purpose: improve from TranslationfromCSV code with new datasets and more ubiquitous code

"""

import numpy as np
import pandas as pd
import re
import os
import statistics 
# import allel

# Change directory to path with all reference/input files
%cd Users/bkearney
    
gene = pd.read_table('gene2ensembl')
human = pd.read_table('Homo_sapiens.gene_info')
mart = pd.read_csv('mart_export.txt',delimiter=',')

# Create Aliases column (delimited Synonyms)
# aliases = human['Synonyms'].tolist()
# aliases_sep = []
# for i in range(len(aliases)):
#     my_list = aliases[i].split("|")
#     my_list.append(human['Symbol'].iloc[i])
#     aliases_sep.append(my_list)
# human['Aliases'] = aliases_sep

# Create geneId column from splicing/filtering dbXrefs
dbXrefs = human['dbXrefs'].tolist()
geneIds = []
for i in range(len(dbXrefs)):
    if 'Ensembl' in dbXrefs[i]:
        gene_id = dbXrefs[i].split("Ensembl:")[1]
    else:
        gene_id = ""
    geneIds.append(gene_id)
human['geneId'] = geneIds


[Errno 2] No such file or directory: 'Users/bkearney'
/Users/bkearney/Summer_20


/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [14]:
#old

def translateList(list_features, testing_df):
    
    test_col = list_features[0]
    reference_df = list_features[1]
    reference_col = list_features[2]
    output_col = list_features[3]

    start_param = testing_df[test_col].tolist()
    in_list = reference_df[reference_col].tolist()

    out_listoflist = []

    for i in range(len(start_param)):
        param = start_param[i]

        match_list=[]
        for j in range(len(in_list)):
            if str(param) in str(in_list[j]):
                index_match = reference_df.iloc[j]
                match_list.append(index_match)

        out_list = []
        for k in range(len(match_list)):
            out_list.append(match_list[k][output_col])

        out_listoflist.append(out_list)
    testing_df['output '+reference_col]=out_listoflist

    return testing_df
#     'geneId (GRCH37.66)','human','geneId','Full_name_from_nomenclature_authority'

def decimal_removal(database, col_index):  # make bp locs into whole numbers for translation
    temp_list = database.iloc[:,col_index].tolist()
    end_list = []
    
    for value in temp_list:
        end_list.append(str(value).split('.',1)[0])
        
    return end_list
    print(end_list)

In [7]:
def f_score(TP,FP,FN):
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_dia = 2 * precision * recall / (precision+recall)
    return f_dia

In [8]:
#old

def scoring(dataF, colname):

    nonmatches_ind = [ind for ind, x in enumerate(dataF[colname]) if len(x)==0 or x != x or x=='-']
    matches_ind = [ind for ind, x in enumerate(dataF[colname]) if len(x)!=0 or x == x or x!='-']
    nonmatches = dataF.iloc[nonmatches_ind]
    matches = dataF.iloc[matches_ind]

 

    truePos = len(dataF)-len(nonmatches)
    falsePos = 0 # How to calculate FP?
    falseNeg = 0
    fScore = f_score(truePos,falsePos,falseNeg)
#     return fScore
    print(len(nonmatches))
    print(len(dataF))
    return round(100*len(nonmatches)/len(dataF),4)


In [73]:
#current function
def combined_translate(input_file, trans_list):
    df = pd.read_excel(input_file)
    
    test_col = trans_list[0]
    reference_df = trans_list[1]
    reference_col = trans_list[2]
    output_col = df.columns[0]
        
    start_param = df[test_col].tolist()
    print(len(start_param))
    in_list = reference_df[reference_col].tolist()
    print(len(in_list))
    out_listoflist = []

    for i in range(len(start_param)):
        param = start_param[i]

        match_list=[]
        for j in range(len(in_list)):
            if str(param) in str(in_list[j]):
                index_match = reference_df.iloc[j]
                match_list.append(index_match)

        out_list = []
        for k in range(len(match_list)):
            out_list.append(match_list[k][output_col])

        out_listoflist.append(out_list)
    out_colname = 'output ' + reference_col
    df[out_colname] = out_listoflist


    nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
    matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
    nonmatches = df.iloc[nonmatches_ind]
    matches = df.iloc[matches_ind]
    print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",test_col)
    print(df[out_colname])

In [ ]:
from bloom_filter import BloomFilter


bloom = BloomFilter(max_elements=10000000)

def matching_check(name):
    if name in bloom:
        return True
    else:
        return False
    
human_modified = human.drop(columns=['GeneID','LocusTag','Modification_date','Feature_type'])
mart_modified = mart.drop(columns=['Gene start (bp)','Gene end (bp)','Transcript start (bp)','Transcript end (bp)'])
    


def add_to_bloom(df):
    for row in df.columns:
        for ele in df[row]:
            if (matching_check(str(ele)) is False):
                bloom.add(str(ele))
            bloom.add(str(ele))
        
add_to_bloom(human_modified)
add_to_bloom(gene)
add_to_bloom(mart)



In [ ]:
def bloom_translate(file):
    df = pd.read_excel(file)
    cols = df.columns
    final_match = []
    
    for col_name in cols:
        match = 0
        miss = 0
        for i in range(len(df[col_name])):
            name = str(df[col_name][i])
            if matching_check(name):
                match=match+1
            else:
                miss = miss+1
        final = match/(match+miss)
        final_match.append(final)
#     print(final_match)
#     print(cols)
    res = {cols[i]: final_match[i] for i in range(len(cols))} 
    res = {key:val for key, val in res.items() if val > 0}

    return res

In [113]:

drug = bloom_translate('drug_supplement.xlsx')
pmc507897 = bloom_translate("/Users/bkearney/Documents/R_projects/pmc_xls/PMC507879/gb-2004-5-8-r54-s1.xls")
print('drug',drug)
print('pmc507897',pmc507897)

drug {'Gene': 0.8928571428571429, 'Chromosomal location': 0.6428571428571429}
pmc507897 {'group_no': 0.012753623188405797, 'annotation': 0.007536231884057971, 'num_unique_peps': 0.9553623188405798, 'tot_num_peps': 0.9043478260869565}
